Initial Setup

In [ ]:
import pandas as pd
from pathlib import Path
from src.data import *

FILE_PATH = Path("data/sample_datasets.xlsx")

data = load_xlsx(FILE_PATH)

customers = data["customers"]
apps = data["applications"]
stores = data["stores"]
marketing = data["marketing"]